<!-- <div align='center' ><font color = blue size='8' face = '黑体'>黄河口悬浮泥沙分布在线反演APP</font></div>
<div align='right' ><font color = green size='2' face = '黑体'>------by xiaoming</font></div>
<div align='left' ><font size='5' face = '楷体'> 
    1.通过云量筛选指定日期黄河口的影像<br>
    2.利用内置的悬沙反演算法将选择的影像执行反演程序，显示反演结果<br>
    3.将反演结果进行简单的制图（指北针、图例、比例尺）<br><br><br>
    界面加载速度受网络影响，请等候.......
    </font></div>                                                                       -->

<!--<div align='center' ><font color = #66CCCC size='6' face = '楷体' ><b>黄河口悬浮泥沙分布在线反演APP</b></font></div>

**可实现:**
<font size='4' face = '楷体'> 
1. 通过云量筛选指定日期黄河口的影像
2. 利用内置的悬沙反演算法将选择的影像执行反演程序，显示反演结果 
3. 制作遥感影像和反演结果的gif动图
4. 增加叶绿素浓度的反演影像显示以及下载
5. 增加海表风速的影像显示

<br>
Contact： Xiaoming  (https://suxiaoming.tech/) 

    
<br>
界面加载速度受网络影响，请稍等...... -->

In [172]:
import os
# os.environ["HTTP_PROXY"]="http://127.0.0.1:4780"
# os.environ["HTTPS_PROXY"]="http://127.0.0.1:4780"

import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
from ipywidgets import *
# import matplotlib.pyplot as plt
# from geemap import cartoee
# from matplotlib.lines import Line2D
# import tkinter as tk
# from tkinter import filedialog
import math
import numpy as np
import pandas as pd

# from tqdm import tqdm
import time
import IPython.display as display
# import win32api,win32con

In [173]:
Map = geemap.Map(center=[37.7,118.8],zoom=9)
m = Map
m.add_basemap()
m.layout.width = '60%'
m.layout.height = '700px'

In [174]:
# 添加landsat卫星影像数据，卫星影像按照云量进行筛选的函数
def cloud_select(year,cloud):
    if 1984<= year <=2011:
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        images = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .filter(ee.Filter.lte('CLOUD_COVER',cloud))

        sys_id = images.aggregate_array('system:id').getInfo()
        cloud = images.aggregate_array('CLOUD_COVER').getInfo()
        # 显示影响的序列号加上云量A
        result = []
        for i in range(len(sys_id)):
            result.append(sys_id[i] + ' : '+ str(cloud[i]))
        return result,sys_id
    elif 2012 <= year < 2014:
        
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .filter(ee.Filter.lte('CLOUD_COVER',cloud))

        sys_id = images.aggregate_array('system:id').getInfo()
        cloud = images.aggregate_array('CLOUD_COVER').getInfo()
        # 显示影响的序列号加上云量
        result = []
        for i in range(len(sys_id)):
            result.append(sys_id[i] + ' : '+ str(cloud[i]))
        return result,sys_id
    else:
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .filter(ee.Filter.lte('CLOUD_COVER',cloud))

        sys_id = images.aggregate_array('system:id').getInfo()
        cloud = images.aggregate_array('CLOUD_COVER').getInfo()
        # 显示影响的序列号加上云量
        result = []
        for i in range(len(sys_id)):
            result.append(sys_id[i] + ' : '+ str(cloud[i]))
        return result,sys_id

In [175]:
# 添加modis卫星影像数据，卫星影像按照云量进行筛选的函数

# Extract MODIS QA information from the "state_1km" QA band
# and use it to mask out cloudy and deep ocean areas.
#
# QA Band information is available at:
# https:#lpdaac.usgs.gov/products/modis_products_table/mod09ga
# Table 1: 1-kilometer State QA Descriptions (16-bit)

#*
 # Returns an image containing just the specified QA bits.
 #
 # Args:
 #   image - The QA Image to get bits from.
 #   start - The first bit position, 0-based.
 #   end   - The last bit position, inclusive.
 #   name  - A name for the output image.
 #

# https://www.jianshu.com/p/abb4ee584385
def getQABits(image, start, end, newName):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start, end, 1):
       pattern += math.pow(2, i)

    return image.select([0], [newName]) \
                  .bitwiseAnd(pattern) \
                  .rightShift(start)

# 添加卫星影像数据，卫星影像按照云量进行筛选的函数
# 设置年月，筛选出一个月的影像数据集
def cloud_select_modis(year,month,cloud_value):
    start_m = ee.Date.fromYMD(year,month,1)
    end_m = start_m.advance(1,'month')
    images = ee.ImageCollection('MODIS/006/MOD09GA')\
               .filterDate(start_m,end_m)
    
    days = images.size().getInfo()
    ic = images.toList(days)
    
    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)
    
    # 设置进度条
    p_bar = tqdm(range(days), desc="MODIS Filtering: ", total=days, ncols=75)
    
    result_modis = []
    modis_id = []
    for i in p_bar:
        time.sleep(0.01)
        #### result,idd = yunliang(ic.get(i))
        # Reference a single MODIS MOD09GA image.
        # image = ee.Image('MODIS/006/MOD09GA/2012_10_11').clip(roi)
        image = ee.Image(ic.get(i)).clip(roi)
        # Select the QA band
        QA = image.select('state_1km')

        # Get the cloud_state bits and find cloudy areas.找云
        cloud = getQABits(QA, 0, 1, 'cloud_state') \
                            .expression("b(0) == 1 || b(0) == 2")
        # Create a mask that filters out cloudy areas.创建无云区
        mask = cloud.Not()

        #Calculate the pixel area in square kilometer 计算云图层中一个像素的面积
        area_pixelcloud = cloud.multiply(ee.Image.pixelArea()).divide(1000*1000)

        #Reducing the statistics for your study area  计算云区的面积
        cloud_area = area_pixelcloud.reduceRegion (
          reducer = ee.Reducer.sum(),
          geometry = roi,
          scale = 30,
          maxPixels = 1e9
        ).getInfo()['cloud_state']
        ca = round(cloud_area)

        #area(10)这个地方参数设置很重要，不然报错，In the documentation for Geometry.area(maxError, proj) says:
        # maxError (ErrorMargin, default: null): The maximum amount of error tolerated when performing any necessary reprojection.
        roi_area = roi.area(10).divide(1e6).round()  # roi的面积并且取整
        cloud_perc = round(ca/int(roi_area.getInfo()),4)
        cloud_percent = format(cloud_perc,".0%")

        id = image.get('system:id').getInfo()
        if int(cloud_perc * 100) < cloud_value:
            result_modis.append('MODIS/006/MODOCGA/' + id[-10:] + ' : '+ str(cloud_percent))
            modis_id.append('MODIS/006/MODOCGA/' + id[-10:]) # 将9GA转为CGA影像
    return result_modis,modis_id
    p_bar.close() # 清理资源

In [176]:
# 反演
# Landsat 7去条带
def func_lvr(image):
  filled1a = image.focal_mean(2, 'square', 'pixels', 1)
  return filled1a.blend(image) 

# 反演SSC
def l5_ssc(image):
    sssc = image.expression(
        '1603.194 * (x*x*x) - 2677.011 * (x*x) + 2569.904 * x -368.906',{
            'x': image.select('B4').divide(image.select('B1'))
        }
    )
    return sssc

def l7_ssc(image):
    sssc = image.expression(
        '1502.239 * (x*x*x) - 2529.413 * (x*x) + 2490.969 * x -358.319',{
            'x': image.select('B4').divide(image.select('B1'))
        }
    )
    return sssc

def l8_ssc(image):
    sssc = image.expression(
        '1406.405 * (x*x*x) - 1824.106 * (x*x) + 2323.982 * x -268.034',{
            'x': image.select('B5').divide(image.select('B2'))
        }
    )
    return sssc

def modis_ssc(image):
    sssc = image.expression(
        '10**(1.106 + 11.035 * math.log(10,(b+c)) - 0.685 * math.log(10,a/b))',{
            'a': image.select('sur_refl_b10'),'b' : image.select('sur_refl_b12'),'c': image.select('sur_refl_b13'),
        }
    )
    return sssc

def l8_chla(image):
    chla = image.expression(
        '6.8 * a*a - 20.77 * a + 17.02',{
            'a':(image.select('B2').add(image.select('B3'))).divide((image.select('B1').add(image.select('B4')))),
    })
    return chla

In [177]:
# 添加卫星影像数据，卫星影像按照云量进行云量最优筛选筛选的函数
def cloud_select_first(year):
    if 1984<= year <=2011:
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        image = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .sort('CLOUD_COVER').first()

        sys_id = image.get('system:id').getInfo()
        
        return sys_id
    elif 2012 <= year <= 2014:
        
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        image = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .sort('CLOUD_COVER').first()
                      
        sys_id = image.get('system:id').getInfo()
        return sys_id
    else:
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .sort('CLOUD_COVER').first()
                      
        sys_id = image.get('system:id').getInfo()
        return sys_id
                      

In [178]:
def ui():
    global out,sscgif_btn,sscgif_btn,imagegif_btn,seletpath_btn,path,speed,\
    title,color,time_range,zero_btn,plot_btn,ssc_btn,image_btn,image_cb,label0,\
    result_ta,static_btn,cloud_slider,time_slider,image_select,output_widget,\
    sc_btn,shp_path,yanzheng_result,yanzheng_btn,exportimg_btn,exportssc_btn,timemonth_slider,export_cb,\
    chla_btn,wind_btn,exportchla_btn,exportwind_btn
    
    
    # 界面设计搭建
    style = {'description_width':'initial'}

    bt_style = dict(
        button_color='#9ACD32',
        font_style='italic',
        font_weight='bold',
        font_variant="small-caps",
        text_color='red',
        text_decoration='underline'
    )
    # 在地图右下角增加一个框用于显示文字或者图表
    output_widget = widgets.Output(layout={'border':'1px solid black'})

    # 软件标题加介绍
    labe00 = HTML(value="<div align='center' ><font color = #32CD32 size='6' face = '华文行楷' ><b>黄河口悬浮泥沙分布在线反演APP</b></font></div>")
    labe11 = HTML(value="<p><font size='2.5' face = '楷体'> 1. 通过云量筛选指定日期黄河口的影像<br />\
                  2. 利用内置的悬沙反演算法将选择的影像执行反演程序，显示反演结果 <br / >\
                  3. 制作遥感影像和反演结果的gif动图<br / >\
                  4. 叶绿素浓度的反演影像以及海表风速的显示以及下载<br / >\
                  Contact： <a href='https://suxiaoming.tech/'>Xiaomingbluesky</a></font></p>")
              
    f0 = widgets.VBox([labe00])
    f00 = widgets.VBox([labe11])

    # 小部件
    image_select = widgets.Dropdown(
        description = '选择数据源：',
        options=['Landsat','Landsat'],
        value = 'Landsat',
        style = style
    )
    image_select.description = "<font size='2' face = '华文细黑'>选择数据源：</font>"

    time_slider = widgets.IntSlider(
        value = 1984,
        min = 1984,
        max = 2020,
        step = 1,
        description = '年份',
        orientation = 'horizontal',
        style = style
    )
    time_slider.description = "<font size='2' face = '华文细黑'>年份</font>"

    timemonth_slider = widgets.IntSlider(
        value = 0,
        min = 1,
        max = 12,
        step = 1,
        description = '月份',
        orientation = 'horizontal',
        style = style,
        disabled = True
    )
    timemonth_slider.description = "<font size='2' face = '华文细黑'>月份</font>"

    cloud_slider = widgets.FloatSlider(
        value = 10,
        min = 0,
        max = 100,
        step = 1,
        description = '云量',
        orientation = 'horizontal',
        style = style,
        readout_format = '.1f'
    )
    cloud_slider.description = "<font size='2' face = '华文细黑'>云量</font>"

    # 添加一个选择框跟底下滑动按钮的联动
    def value_changed(change):
        if  change.new == 'Landsat':
            time_slider.min = 1984
            time_slider.max = 2020
            timemonth_slider.disabled = True
        else:
            time_slider.min = 2001
            time_slider.max = 2020
            timemonth_slider.disabled = False

    image_select.observe(value_changed, 'value')


    static_btn = widgets.Button(
        description = '按云量筛选',
        button_style = 'primary',
        tooltip = '单击',
        style = bt_style
    )

    #     shp_path = widgets.Text(
    #         value='',
    #         placeholder='',
    #         description='',
    #         disabled=False   
    #     )

    #     label1 = widgets.Label(
    #         value='反演结果验证：'
    #     )

    #     label2 = widgets.Label(
    #         value='精度验证结果：'
    #     )

    label0 = HTML(value="结果（eg. id : cloud_cover）")
    shp_path = Text(description='', layout=Layout(width='450px'),style = style)
    yanzheng_result = Textarea(description='', layout=Layout(width='300px', height='120px'),style = style)
    result_ta = Textarea(description='', layout=Layout(width='100%', height='auto'),style = style)

    image_cb = Combobox(description='选择影像', layout=Layout(width='400px', height='auto'),style = style)
    image_cb.description = "<font size='2' face = '华文细黑'>选择影像</font>"

    image_btn = Button(description='显示卫星影像', layout=Layout(width='auto',height='auto') ,style = bt_style,button_style = 'primary')
    image_btn.style.button_color = '#F4A460'

    ssc_btn = Button(description='显示反演SSC', layout=Layout(width='auto',height='auto') ,style = bt_style,button_style = 'primary')
    #   plot_btn = Button(description='制图', layout=Layout(width='auto',height='auto'), style=ButtonStyle(button_color='#88EA79'))
    ssc_btn.style.button_color = '#66CDAA'

    zero_btn = Button(description='软件初始化', layout=Layout(width='auto',height='auto'), style = bt_style,button_style = 'primary')
    zero_btn.style.button_color = '#FFD700'

    chla_btn = Button(description='显示反演chla', layout=Layout(width='auto',height='auto') ,style = bt_style,button_style = 'primary')
    chla_btn.style.button_color = '#66CDAA'

    wind_btn = Button(description='显示风速影像', layout=Layout(width='auto',height='auto') ,style = bt_style,button_style = 'primary')
    wind_btn.style.button_color = '#66CDAA'


    #   sc_btn = Button(description='导入实测点数据（.shp）', layout=Layout(width='auto',height='auto') ,style=ButtonStyle(button_color='#69A6FF'))
    #   yanzheng_btn = Button(description='精度验证', layout=Layout(width='auto',height='auto') ,style=ButtonStyle(button_color='#69A6FF'))
    exportimg_btn = Button(description='导出原始影像', layout=Layout(width='auto',height='auto') ,style = bt_style)
    exportssc_btn = Button(description='导出ssc反演图像', layout=Layout(width='auto',height='auto') ,style = bt_style)
    exportchla_btn = Button(description='导出chla反演图像', layout=Layout(width='auto',height='auto') ,style = bt_style)
    exportwind_btn = Button(description='导出风速图像', layout=Layout(width='auto',height='auto') ,style = bt_style)

    export_cb = widgets.Checkbox(
        value=False,
        description="<font size='1.5' face = '微软雅黑'>显示影像的同时导出</font>",
        disabled=False,
        indent=False,
        layout={'width': '150px'}
    )



    # 3
    time_range = widgets.IntRangeSlider(
        value=[1984, 2021],
        min=1984,
        max=2021,
        step=1,
        description='选择年份范围',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
    time_range.description = "<font size='2' face = '华文细黑'>选择年份范围</font>"

    color = widgets.ColorPicker(
        concise=False,
        description='字体颜色：',
        value='white',
        disabled=False
    )
    color.description = "<font size='2' face = '华文细黑'>字体颜色：</font>"

    title = widgets.Text(
        value='Hello World',
        placeholder='Type something',
        description='标题(en)：',
        disabled=False   
    )
    title.description = "<font size='2' face = '华文细黑'>标题(en)：</font>"

    speed = widgets.BoundedIntText(
        value=1,
        min=1,
        max=10,
        step=1,
        description='播放速度：',
        disabled=False
    )
    speed.description = "<font size='2' face = '华文细黑'>播放速度：</font>"

    #     path = widgets.Text(
    #         value='',
    #         placeholder='',
    #         description='',
    #         disabled=False   
    #     )

    #     seletpath_btn = Button(description='选择保存位置', layout=Layout(width='auto',height='auto'), style=ButtonStyle(button_color='#E69494'))
    imagegif_btn = Button(description='卫星影像GIF', layout=Layout(width='20%',height='auto'), style = bt_style,button_style = 'primary')
    sscgif_btn = Button(description='SSC-GIF', layout=Layout(width='20%',height='auto'), style = bt_style,button_style = 'primary')
    chlagif_btn = Button(description='chla-GIF', layout=Layout(width='20%',height='auto'), style = bt_style,button_style = 'primary')
    windgif_btn = Button(description='风速GIF', layout=Layout(width='20%',height='auto'), style = bt_style,button_style = 'primary')

    # 布局

    # 影响筛选
    f1 = widgets.VBox([
        widgets.HBox([image_select]),
        widgets.HBox([time_slider,timemonth_slider]),
        widgets.HBox([cloud_slider,static_btn]),
        widgets.VBox([
            widgets.HBox([label0]),
            widgets.HBox([result_ta])
            ],layout=Layout(
        border='0px solid green', padding='0px',
        align_items='stretch', width='auto'))
        ], layout=Layout(
        border='0px solid gray', padding='0px',
        align_items='stretch', width='auto'))

    # 影像反演
    f2 = widgets.VBox([
        widgets.HBox([image_cb,image_btn]),
        widgets.HBox([export_cb,ssc_btn,chla_btn,wind_btn,zero_btn])
    ], layout=Layout(
        border='0px solid gray', padding='0px',
        align_items='unset', width='auto'))

    # 变化动态图示
    f3 = widgets.VBox([
        widgets.VBox([time_range,color,title,speed]),
        widgets.HBox([imagegif_btn,sscgif_btn,chlagif_btn,windgif_btn], layout=Layout(
        align_items='center')
    )], layout=Layout(
        border='0px solid black', padding='0px',
        align_items='unset', width='auto'))

    # 将输出放在out中，便于清除
    out = widgets.Output(layout={'border': '0px solid black'})
    # with out:
    #     display(f1,f2)
    # out
    t = Tab(children=[f2, f3], _titles={0: '影像反演 & 显示', 1: '变化动态图示'})

    t1 = widgets.VBox([f2,f3])

    labe01 = HTML(value="<font color = #0000FF size='2' face = '楷书' ><b>——>影像筛选</b></font>")
    labe02 = HTML(value="<font color = #0000FF size='2' face = '楷书' ><b>——>影像反演 & 显示</b></font>")
    labe03 = HTML(value="<font color = #0000FF size='2' face = '楷书' ><b>——>变化动态图示</b></font>")

    line = HTML(value="<div style='border:3px double #000000'> </div>")

    t2 = GridBox(children=[f0,f00,line,labe01,f1,labe02,f2,labe03,f3],
                layout=Layout(
                    grid_template_rows='5% 20% 2% 5% 25% 5% 12% 5% 25%')
               )
    f3 = widgets.HBox([t2,m]
    , layout=Layout(
        border='0px solid gray', padding='0px'))
    return f3

In [179]:
bj = ui()
bj

In [180]:
# 点击事件
# 云量筛选
def submit_clicked(a):
#     with output_widget:
# #         output_widget.clear_output()
#         print('请稍等，正在查找中.......')
#         Map.default_style = {'cursor': 'wait'}
    
    try:
        image = image_select.value
        year = time_slider.value
        month = timemonth_slider.value
        cloud = cloud_slider.value
        output_widget.clear_output()
        if image == 'Landsat':
            result_ta.value = ''
            result,sys_id = cloud_select(year,cloud)
            result_ta.value = str(result)
            image_cb.options = sys_id
        else:
            result_ta.value = ''
            result_modis,modis_id = cloud_select_modis(year,month,cloud)
            result_ta.value = str(result_modis)
            image_cb.options = modis_id



    except Exception as e:
        print(e)
        print('查找过程出现错误，请重新输入参数！')
#         Map.default_style = {'cursor': 'default'}


# 显示影像
def show_image(b):
    global image
    with output_widget:
        output_widget.clear_output()
        try:
            id = image_cb.value
            image = ee.Image(id)
            if id[0] == 'L':
                # 判断是不是L7影像，进行去条带操作
                if id[11] == '7':
                    image = func_lvr(image)

                vis_params = {
                    'bands':['B4','B3','B2'],
                    'min': 0,
                    'max': 3000
                }   
                Map.addLayer(image,vis_params,'select landsat image')
                
                if export_cb.value == True:

                    out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                    filename = os.path.join(out_dir, 'landsat.tif')

                    geemap.ee_export_image(image, filename=filename, scale=200, region=roi, file_per_band=False)
            else:
                
                # roi直接用黄河口边界
                roi = ee.Geometry.Polygon(
                    [[[118.0589, 38.4364],
                      [120.2327, 38.1181],
                        [119.7274, 36.5238],
                        [117.5964, 36.8278],
                        [118.0589, 38.4364]]], None, False)
                image = image.clip(roi)
                vis_params = {
                    'bands':['sur_refl_b11','sur_refl_b10','sur_refl_b09'],
                    'min': 0,
                    'max': 2000
                }   
                Map.addLayer(image,vis_params,'select modis image')
                
                    
                
        except Exception as e:
            print(e)
            print('加载失败')
        
        
# 显示SSC
def show_ssc(c):
    global ssc_intial
    with output_widget:
        output_widget.clear_output()
        try:
            if image_cb.value[0] == 'L':
                id = image_cb.value
                image = ee.Image(id)
                #判断是不是L7影像，进行去条带操作
                if id[11] == '7':
                    image = func_lvr(image)

                # 岸线变迁考虑在内
                land_path = 'users/bluesky666/yrd_ssc_coastline/' + str(time_slider.value) + 'land'
                ocean_path = 'users/bluesky666/yrd_ssc_coastline/' + str(time_slider.value) + 'sea'

                landshp = ee.FeatureCollection(land_path)
                oceanshp = ee.FeatureCollection(ocean_path)
                # 利用陆地和海洋shp分别裁剪出陆地和海洋范围
                land = image.clip(landshp)
                ocean = image.clip(oceanshp)

                # Set visualization parameters.
                vis_params0 = {'bands': ['B1'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 1.0}
                vis_params = {'bands': ['constant'], 'palette': ['ff0000', ' ff0600', ' ff0c00', ' ff1200', ' ff1800', ' ff1e00', ' ff2300', ' ff2900', ' ff2f00', ' ff3500', ' ff3b00', ' ff4100', ' ff4700', ' ff4d00', ' ff5300', ' ff5900', ' ff5f00', ' ff6400', ' ff6a00', ' ff7000', ' ff7600', ' ff7c00', ' ff8200', ' ff8800', ' ff8e00', ' ff9400', ' ff9a00', ' ff9f00', ' ffa500', ' ffab00', ' ffb100', ' ffb700', ' ffbd00', ' ffc300', ' ffc900', ' ffcf00', ' ffd500', ' ffdb00', ' ffe000', ' ffe600', ' ffec00', ' fef100', ' fcf500', ' faf900', ' f8fd00', ' f4ff00', ' eeff00', ' e8ff00', ' e2ff00', ' ddff00', ' d7ff00', ' d1ff00', ' cbff00', ' c5ff00', ' bfff00', ' b9ff00', ' b3ff00', ' adff00', ' a7ff00', ' a2ff00', ' 9cff00', ' 96ff00', ' 90ff00', ' 8aff00', ' 84ff00', ' 7eff00', ' 78ff00', ' 72ff00', ' 6cff00', ' 66ff00', ' 61ff00', ' 5bff00', ' 55ff00', ' 4fff00', ' 49ff00', ' 43ff00', ' 3dff00', ' 37ff00', ' 31ff00', ' 2bff00', ' 25ff00', ' 20ff00', ' 1aff00', ' 14ff00', ' 0eff00', ' 08ff00', ' 06ff04', ' 04ff08', ' 02ff0c', ' 00ff10', ' 00ff16', ' 00ff1b', ' 00ff21', ' 00ff27', ' 00ff2d', ' 00ff33', ' 00ff39', ' 00ff3f', ' 00ff45', ' 00ff4b', ' 00ff51', ' 00ff57', ' 00ff5c', ' 00ff62', ' 00ff68', ' 00ff6e', ' 00ff74', ' 00ff7a', ' 00ff80', ' 00ff86', ' 00ff8c', ' 00ff92', ' 00ff97', ' 00ff9d', ' 00ffa3', ' 00ffa9', ' 00ffaf', ' 00ffb5', ' 00ffbb', ' 00ffc1', ' 00ffc7', ' 00ffcd', ' 00ffd3', ' 00ffd8', ' 00ffde', ' 00ffe4', ' 00ffea', ' 00fff0', ' 00fff6', ' 00fffc', ' 00fcff', ' 00f6ff', ' 00f0ff', ' 00eaff', ' 00e5ff', ' 00dfff', ' 00d9ff', ' 00d3ff', ' 00cdff', ' 00c7ff', ' 00c1ff', ' 00bbff', ' 00b5ff', ' 00afff', ' 00aaff', ' 00a4ff', ' 009eff', ' 0098ff', ' 0092ff', ' 008cff', ' 0086ff', ' 0080ff', ' 007aff', ' 0074ff', ' 006eff', ' 0069ff', ' 0063ff', ' 005dff', ' 0057ff', ' 0051ff', ' 004bff', ' 0045ff', ' 003fff', ' 0039ff', ' 0033ff', ' 002dff', ' 0028ff', ' 0022ff', ' 001cff', ' 0016ff', ' 0010ff', ' 020cff', ' 0408ff', ' 0604ff', ' 0800ff', ' 0e00ff', ' 1300ff', ' 1900ff', ' 1f00ff', ' 2500ff', ' 2b00ff', ' 3100ff', ' 3700ff', ' 3d00ff', ' 4300ff', ' 4900ff', ' 4f00ff', ' 5400ff', ' 5a00ff', ' 6000ff', ' 6600ff', ' 6c00ff', ' 7200ff', ' 7800ff', ' 7e00ff', ' 8400ff', ' 8a00ff', ' 9000ff', ' 9500ff', ' 9b00ff', ' a100ff', ' a700ff', ' ad00ff', ' b300ff', ' b900ff', ' bf00ff', ' c500ff', ' cb00ff', ' d000ff', ' d600ff', ' dc00ff', ' e200ff', ' e800ff', ' ee00ff', ' f400ff', ' f800fd', ' fa00f9', ' fc00f5', ' fe00f1', ' ff00ed', ' ff00e7', ' ff00e1', ' ff00db', ' ff00d5', ' ff00cf', ' ff00c9', ' ff00c3', ' ff00bd', ' ff00b7', ' ff00b1', ' ff00ac', ' ff00a6', ' ff00a0', ' ff009a', ' ff0094', ' ff008e', ' ff0088', ' ff0082', ' ff007c', ' ff0076', ' ff0071', ' ff006b', ' ff0065', ' ff005f', ' ff0059', ' ff0053', ' ff004d', ' ff0047', ' ff0041', ' ff003b', ' ff0035', ' ff0030', ' ff002a', ' ff0024', ' ff001e', ' ff0018'], 'min': 0.0, 'max': 3000.0, 'opacity': 1.0}

                #判断是哪个传感器，用不同的反演公式
                if id[11] == '5':
                    # ssc的原始数据
                    ssc_intial = l5_ssc(ocean)
                    ssc = l5_ssc(ocean).visualize(**vis_params)

                elif id[11] == '7':
                    # ssc的原始数据
                    ssc_intial = l7_ssc(ocean)
                    ssc = l7_ssc(ocean).visualize(**vis_params)

                elif id[11] == '8':
                    # ssc的原始数据
                    ssc_intial = l8_ssc(ocean)
                    ssc = l8_ssc(ocean).visualize(**vis_params)

                Map.addLayer(ssc,{},'ssc')
                Map.addLayer(ssc_intial,{},'ssc_intial')

                # 将陆地设置为0，方便显示
                land = land.where(land,0)
                land = land.visualize(**vis_params0)
                Map.addLayer(land,{},'land')

                #为了便于成图陆地海洋合并成一张图             
                global blend 
                blend = land.blend(ssc)
                Map.addLayer(blend,{},'blend')
                
            else:
                ####警告信息框
                win32api.MessageBox(0, "暂时无法使用！", "提醒",win32con.MB_ICONWARNING)
        except Exception as e:
            print(e)
            print('加载失败')
            
        if export_cb.value == True:
            exssc(a4)


# 显示反演chla
def show_chla(c):
    global chla_intial
    with output_widget:
        output_widget.clear_output()
        try:
            if image_cb.value[0] == 'L':
                id = image_cb.value
                image = ee.Image(id)
                #判断是不是L7影像，进行去条带操作
                if id[11] == '7':
                    image = func_lvr(image)

                # 岸线变迁考虑在内
                land_path = 'users/bluesky666/yrd_ssc_coastline/' + str(time_slider.value) + 'land'
                ocean_path = 'users/bluesky666/yrd_ssc_coastline/' + str(time_slider.value) + 'sea'

                landshp = ee.FeatureCollection(land_path)
                oceanshp = ee.FeatureCollection(ocean_path)
                # 利用陆地和海洋shp分别裁剪出陆地和海洋范围
                land = image.clip(landshp)
                ocean = image.clip(oceanshp)

                # Set visualization parameters.
                vis_params0 = {'bands': ['B1'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 1.0}
                vis_params = {'bands': ['constant'], 'palette': ['ff0000', ' ff0600', ' ff0c00', ' ff1200', ' ff1800', ' ff1e00', ' ff2300', ' ff2900', ' ff2f00', ' ff3500', ' ff3b00', ' ff4100', ' ff4700', ' ff4d00', ' ff5300', ' ff5900', ' ff5f00', ' ff6400', ' ff6a00', ' ff7000', ' ff7600', ' ff7c00', ' ff8200', ' ff8800', ' ff8e00', ' ff9400', ' ff9a00', ' ff9f00', ' ffa500', ' ffab00', ' ffb100', ' ffb700', ' ffbd00', ' ffc300', ' ffc900', ' ffcf00', ' ffd500', ' ffdb00', ' ffe000', ' ffe600', ' ffec00', ' fef100', ' fcf500', ' faf900', ' f8fd00', ' f4ff00', ' eeff00', ' e8ff00', ' e2ff00', ' ddff00', ' d7ff00', ' d1ff00', ' cbff00', ' c5ff00', ' bfff00', ' b9ff00', ' b3ff00', ' adff00', ' a7ff00', ' a2ff00', ' 9cff00', ' 96ff00', ' 90ff00', ' 8aff00', ' 84ff00', ' 7eff00', ' 78ff00', ' 72ff00', ' 6cff00', ' 66ff00', ' 61ff00', ' 5bff00', ' 55ff00', ' 4fff00', ' 49ff00', ' 43ff00', ' 3dff00', ' 37ff00', ' 31ff00', ' 2bff00', ' 25ff00', ' 20ff00', ' 1aff00', ' 14ff00', ' 0eff00', ' 08ff00', ' 06ff04', ' 04ff08', ' 02ff0c', ' 00ff10', ' 00ff16', ' 00ff1b', ' 00ff21', ' 00ff27', ' 00ff2d', ' 00ff33', ' 00ff39', ' 00ff3f', ' 00ff45', ' 00ff4b', ' 00ff51', ' 00ff57', ' 00ff5c', ' 00ff62', ' 00ff68', ' 00ff6e', ' 00ff74', ' 00ff7a', ' 00ff80', ' 00ff86', ' 00ff8c', ' 00ff92', ' 00ff97', ' 00ff9d', ' 00ffa3', ' 00ffa9', ' 00ffaf', ' 00ffb5', ' 00ffbb', ' 00ffc1', ' 00ffc7', ' 00ffcd', ' 00ffd3', ' 00ffd8', ' 00ffde', ' 00ffe4', ' 00ffea', ' 00fff0', ' 00fff6', ' 00fffc', ' 00fcff', ' 00f6ff', ' 00f0ff', ' 00eaff', ' 00e5ff', ' 00dfff', ' 00d9ff', ' 00d3ff', ' 00cdff', ' 00c7ff', ' 00c1ff', ' 00bbff', ' 00b5ff', ' 00afff', ' 00aaff', ' 00a4ff', ' 009eff', ' 0098ff', ' 0092ff', ' 008cff', ' 0086ff', ' 0080ff', ' 007aff', ' 0074ff', ' 006eff', ' 0069ff', ' 0063ff', ' 005dff', ' 0057ff', ' 0051ff', ' 004bff', ' 0045ff', ' 003fff', ' 0039ff', ' 0033ff', ' 002dff', ' 0028ff', ' 0022ff', ' 001cff', ' 0016ff', ' 0010ff', ' 020cff', ' 0408ff', ' 0604ff', ' 0800ff', ' 0e00ff', ' 1300ff', ' 1900ff', ' 1f00ff', ' 2500ff', ' 2b00ff', ' 3100ff', ' 3700ff', ' 3d00ff', ' 4300ff', ' 4900ff', ' 4f00ff', ' 5400ff', ' 5a00ff', ' 6000ff', ' 6600ff', ' 6c00ff', ' 7200ff', ' 7800ff', ' 7e00ff', ' 8400ff', ' 8a00ff', ' 9000ff', ' 9500ff', ' 9b00ff', ' a100ff', ' a700ff', ' ad00ff', ' b300ff', ' b900ff', ' bf00ff', ' c500ff', ' cb00ff', ' d000ff', ' d600ff', ' dc00ff', ' e200ff', ' e800ff', ' ee00ff', ' f400ff', ' f800fd', ' fa00f9', ' fc00f5', ' fe00f1', ' ff00ed', ' ff00e7', ' ff00e1', ' ff00db', ' ff00d5', ' ff00cf', ' ff00c9', ' ff00c3', ' ff00bd', ' ff00b7', ' ff00b1', ' ff00ac', ' ff00a6', ' ff00a0', ' ff009a', ' ff0094', ' ff008e', ' ff0088', ' ff0082', ' ff007c', ' ff0076', ' ff0071', ' ff006b', ' ff0065', ' ff005f', ' ff0059', ' ff0053', ' ff004d', ' ff0047', ' ff0041', ' ff003b', ' ff0035', ' ff0030', ' ff002a', ' ff0024', ' ff001e', ' ff0018'], 'min': 0.0, 'max': 3000.0, 'opacity': 1.0}

                chla_intial = l8_chla(ocean)
                chla = l8_chla(ocean).visualize(**vis_params)

                Map.addLayer(chla,{},'chla')
                Map.addLayer(chla_intial,{},'chla_intial')

                # 将陆地设置为0，方便显示
                land = land.where(land,0)
                land = land.visualize(**vis_params0)
                Map.addLayer(land,{},'land')

                #为了便于成图陆地海洋合并成一张图             
                global blend 
                blend = land.blend(chla)
                Map.addLayer(blend,{},'blend')
            else:
                ####警告信息框
                win32api.MessageBox(0, "暂时无法使用！", "提醒",win32con.MB_ICONWARNING)
        except Exception as e:
            print(e)
            print('加载失败')
            
# 显示风速
def show_wind(c):
    global wind_intial
    with output_widget:
        output_widget.clear_output()
        try:
            id = image_cb.value
            year = id[-8:-4]
            month = id[-4:-2]
            day = id[-2:]
            start_year = ee.Date.fromYMD(int(year),int(month),int(day))
            end_year = start_year.advance(1,'day')
            
            # 日均风场数据
            u_wind = ee.ImageCollection("ECMWF/ERA5/DAILY")\
                        .select('u_component_of_wind_10m')\
                        .filterDate(start_year,end_year)\
                        .first()

            v_wind = ee.ImageCollection("ECMWF/ERA5/DAILY")\
                        .select('v_component_of_wind_10m')\
                        .filterDate(start_year,end_year)\
                        .first()

            visualization = {
              'min': -10.0,
              'max': 10.0,
              'palette': [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
              ]
            }

            # 岸线变迁考虑在内
            land_path = 'users/bluesky666/yrd_ssc_coastline/' + str(time_slider.value) + 'land'
            ocean_path = 'users/bluesky666/yrd_ssc_coastline/' + str(time_slider.value) + 'sea'

            landshp = ee.FeatureCollection(land_path)
            oceanshp = ee.FeatureCollection(ocean_path)
            
            
            roi = ee.Geometry.Polygon(
                    [[[118.0589, 38.4364],
                      [120.2327, 38.1181],
                        [119.7274, 36.5238],
                        [117.5964, 36.8278],
                        [118.0589, 38.4364]]], None, False)
            # u_wind
            # 利用陆地和海洋shp分别裁剪出陆地和海洋范围
            land = u_wind.clip(roi).clip(landshp)
            ocean = u_wind.clip(roi).clip(oceanshp)

            
            # Set visualization parameters.
            vis_params0 = {'bands': ['u_component_of_wind_10m'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 1.0}
        
            u_wind_intial = u_wind.clip(roi)
            u_wind = u_wind.clip(roi).visualize(**visualization)

            Map.addLayer(u_wind,{},'u_wind')
#             Map.addLayer(u_wind_intial,{},'u_wind_intial')
            
            # v_wind
            ocean = v_wind.clip(oceanshp)
        
            v_wind_intial = v_wind.clip(roi)
            v_wind = v_wind.clip(roi).visualize(**visualization)

            Map.addLayer(v_wind,{},'v_wind')
#             Map.addLayer(v_wind_intial,{},'v_wind_intial')
            
            # 将陆地设置为0，方便显示
            land = land.where(land,0)
            land = land.visualize(**vis_params0)
            Map.addLayer(land,{},'land')

#             #为了便于成图陆地海洋合并成一张图             
#             global blend 
#             blend = land.blend(chla)
#             Map.addLayer(blend,{},'blend')
        except Exception as e:
            print(e)
            print('加载失败')
            
# 制图
def plt_show(d):
    fig = plt.figure(figsize=(15, 10),dpi = 300)

    # here is the bounding box of the map extent we want to use
    # formatted a [W,S,E,N]
    zoom_region = [117.4714, 36.4558, 120.4, 38.4966]

    # plot the map over the region of interest
    vis = {}
    ax = cartoee.get_map(blend, vis_params=vis, region=zoom_region)

    # add the gridlines and specify that the xtick labels be rotated 45 degrees
    cartoee.add_gridlines(ax, interval=0.3, xtick_rotation=0, linestyle=":")

    # add coastline
    # ax.coastlines(color="cyan")

    # add a colorbar to the map using the visualization params we passed to the map
    vis_params_zq = {'palette': ['4368AF', '6991CC', '4188C8', '33BCF0', '87CEEC', '70C7DA', '98D1BE', '6FBD69', '54B33F', '9AC834', 'BDB769', 'F0E78A', 'FED507', 'DAA41D', 'D16A28', 'EE7D53', 'CE5C5C', 'E84A27', 'E6212A', 'B02F33', '792A2D'], 'min': 0.0, 'max': 5000.0, 'opacity': 1.0}
    cartoee.add_colorbar(ax, vis_params_zq, loc="right", label = "SSC(mg/L)",orientation="vertical")

    # add north arrow
    cartoee.add_north_arrow(ax, text="N", xy=(0.1, 0.95), text_color="black", arrow_color="white", fontsize=20)

    # add scale bar
    cartoee.add_scale_bar_lite(ax, length=30, xy=(0.1, 0.05), fontsize=20, color="black", unit="km")

    ax.set_title(label = "Distribution of Suspended Sediment Concentration in the Yellow River estuary on "+ image_cb.value[-8:] , fontsize=15)

    # add legend
    # legend_elements = [Line2D([], [], color='#00ffff', lw=2, label='Coastline'),
    #     Line2D([], [], marker='o', color='#A8321D', label='City', markerfacecolor='#A8321D', markersize=10, ls ='')]

    # cartoee.add_legend(ax, legend_elements, loc='lower right')
    plt.show()

# 回归评估指标计算，平均绝对误差，均方误差，均方根误差，平均相对误差
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=sum(absError)/len(absError)
    return mae,mse,RMSE

def compute_rsquared(X, Y):
    xBar = np.mean(X)
    yBar = np.mean(Y)
    SSR = 0
    varX = 0
    varY = 0
    for i in range(0, len(X)):
        diffXXBar = X[i] - xBar
        diffYYBar = Y[i] - yBar
        SSR += (diffXXBar * diffYYBar)
        varX += diffXXBar ** 2
        varY += diffYYBar ** 2

    SST = math.sqrt(varX * varY)
    r2=round((SSR / SST) ** 2,3)
    return r2

# 平均相对误差
def mean_relative_error(y_true, y_pred):
    relative_error = np.average(np.abs(y_true - y_pred) / y_true, axis=0)
    return relative_error

# 导入实测数据
def data_import(a1):
    global out_csv
    # 将实测数据对应点的预测值提取出来csv
    '''打开选择文件夹对话框'''
    root = tk.Tk()
    root.withdraw()

    # Folderpath = filedialog.askdirectory() #获得选择好的文件夹
    Filepath = filedialog.askopenfilename(title='选择实测数据的点shp文件', filetypes=[('SHP', '*.shp'), ('All Files', '*')]) #获得选择好的文件
    in_shp = Filepath
    # work_dir = os.path.expanduser('~/Downloads')
    # in_shp = os.path.join(work_dir, 'us_cities.shp')
    # if not os.path.exists(in_shp):
    #     data_url = 'https://github.com/giswqs/data/raw/main/us/us_cities.zip'
    #     geemap.download_from_url(data_url, out_dir=work_dir )
    shp_path.value = Filepath
    in_fc = geemap.shp_to_ee(in_shp)
    Map.addLayer(in_fc, {}, 'meature')

    #  Folderpath = filedialog.askdirectory() #获得选择好的文件夹
    # 默认将csv存在默认文件夹下
    Folderpath = os.path.expanduser('~/Downloads')
    work_dir = Folderpath
    out_csv = os.path.join(work_dir, 'point.csv')
    geemap.extract_values_to_points(in_fc, ssc_intial, out_csv)

# 精度验证
def yanzheng(a2):
    data = pd.read_csv(out_csv)
    col_1 = data["z"]  #获取一列，用一维数据
    testX = np.array(col_1)
    col_12 = data["constant"]  #获取di两列
    testY = np.array(col_12)
    
    mae,mse,rmse=compute_mae_mse_rmse(testX,testY)
    r2=compute_rsquared(testX,testY)
    mre = mean_relative_error(testX,testY)
    
    yz_result = "R²(决定系数)：%.4f " %r2  + '\n' + "MAE(平均绝对误差):%.4f " %mae + '\n'  + "MRE(平均相对误差):%.4f "%mre + '\n' + "MSE(均方误差):%.4f " %mse + '\n'  + "RMSE(均方根误差):%.4f "%rmse 
    
    yanzheng_result.value = yz_result

# 下载导出遥感影像
def eximg(a3):
    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)
    
#     '''打开选择文件夹对话框'''
#     root = tk.Tk()
#     root.withdraw()

#     # Folderpath = filedialog.askdirectory() #获得选择好的文件夹
#     Filepath = filedialog.asksaveasfilename(title='保存遥感影像', filetypes=[('TIF', '*.tif'), ('All Files', '*')]) #获得选择好的文件
#     filename = Filepath + '.tif'
    
    out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    filename = os.path.join(out_dir, 'landsat.tif')

    geemap.ee_export_image(image, filename=filename, scale=200, region=roi, file_per_band=False)
    
# 下载导出反演影像
def exssc(a3):
    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)
    
#     '''打开选择文件夹对话框'''
#     root = tk.Tk()
#     root.withdraw()

#     # Folderpath = filedialog.askdirectory() #获得选择好的文件夹
#     Filepath = filedialog.asksaveasfilename(title='保存反演影像', filetypes=[('TIF', '*.tif'), ('All Files', '*')]) #获得选择好的文件
#     filename = Filepath + '.tif'

    out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    filename = os.path.join(out_dir, 'ssc.tif')
    
    geemap.ee_export_image(ssc_intial, filename=filename, scale=110, region=roi, file_per_band=False)
    
    
# 下载导出反演影像
def exchla(a3):
    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)
    
#     '''打开选择文件夹对话框'''
#     root = tk.Tk()
#     root.withdraw()

#     # Folderpath = filedialog.askdirectory() #获得选择好的文件夹
#     Filepath = filedialog.asksaveasfilename(title='保存反演影像', filetypes=[('TIF', '*.tif'), ('All Files', '*')]) #获得选择好的文件
#     filename = Filepath + '.tif'

    out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    filename = os.path.join(out_dir, 'chla.tif')
    
    geemap.ee_export_image(chla_intial, filename=filename, scale=110, region=roi, file_per_band=False)
    
# 地图初始化
def map_zero(e):
    time_slider.value = 1984
    cloud_slider.value = 10
    result_ta.value = ""
    image_cb.value = ""
    yanzheng_result.value =""
    image_cb.options = []
    Map.clear_layers()
    Map.add_basemap('Google Maps')
    out.clear_output()
#     bj.close()
#     tt = ui()
#     tt
    
    
# 选择保存路径
def select_path(f):
    '''打开选择文件夹对话框'''
    root = tk.Tk()
    root.withdraw()

    Folderpath = filedialog.askdirectory() #获得选择好的文件夹
    # Filepath = filedialog.askopenfilename() #获得选择好的文件
    work_dir = Folderpath
    path.value = work_dir

#影像GIF
def imagegif(g):
    fontcolor = color.value
    title0 = title.value
    speed0 = speed.value
    
    out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#     filename = os.path.join(out_dir, 'landsat.tif')
    work_dir = out_dir   # 加上默认的路径
    start_year = time_range.value[0]
    end_year = time_range.value[1]
    # 制作影像动态图
    images = []  #用来存储所有的影像id
    # 用影像Id将所有影像合成一个collection
    for i in range(start_year,end_year):
        sys_id = cloud_select_first(i)
        image = ee.Image(sys_id)
        # 判断是不是L7影像，进行去条带操作
        if sys_id[11] == '7':
            image = func_lvr(image)
        images.append(image)
    collection = ee.ImageCollection(images)

    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)

    # 定义动图参数
    video_args = {
        'dimensions': 600,
        'region': roi,
        'framePerSecond': speed0,
        'bands': ['B4','B3','B2'],
        'min': 0,
        'max': 4000,
        'gamma': [1,1,1]
    }

    vv = collection.getVideoThumbURL(video_args)
    geemap.download_from_url(vv, out_dir=work_dir, unzip = False, out_file_name = "test00.gif")

    out_gif = os.path.join(work_dir, "test00.gif")

    geemap.add_text_to_gif(out_gif, out_gif, xy=('3%', '5%'), text_sequence=start_year, font_size=30, font_color=fontcolor, add_progress_bar=False)

    label = title0
    geemap.add_text_to_gif(out_gif, out_gif, xy=('2%', '88%'), text_sequence=label, font_size=30, font_color=fontcolor, progress_bar_color='cyan')

#     geemap.show_image(out_gif)
    with out:
        geemap.show_image(out_gif)
    out

#SSC-GIF
def sscgif(h):
    fontcolor = color.value
    title0 = title.value
    speed0 = speed.value
    out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#     filename = os.path.join(out_dir, 'landsat.tif')
    work_dir = out_dir   # 加上默认的路径
    start_year = time_range.value[0]
    end_year = time_range.value[1]
    # 制作SSC动态图
    SSCs = []  #用来存储所有的影像id
    # 用影像Id将所有影像合成一个collection
    for i in range(start_year,end_year):
        sys_id = cloud_select_first(i)
        image = ee.Image(sys_id)
        # 判断是不是L7影像，进行去条带操作
        if sys_id[11] == '7':
            image = func_lvr(image)

            
        # 岸线变迁考虑在内
        land_path = 'users/bluesky666/yrd_ssc_coastline/' + str(i) + 'land'
        ocean_path = 'users/bluesky666/yrd_ssc_coastline/' + str(i) + 'sea'
        
        landshp = ee.FeatureCollection(land_path)
        oceanshp = ee.FeatureCollection(ocean_path)
        
        # 利用陆地和海洋shp分别裁剪出陆地和海洋范围
        land = image.clip(landshp)
        ocean = image.clip(oceanshp)

        # Set visualization parameters.
        vis_params0 = {'bands': ['B1'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 1.0}
        vis_params = {'bands': ['constant'], 'palette': ['ff0000', ' ff0600', ' ff0c00', ' ff1200', ' ff1800', ' ff1e00', ' ff2300', ' ff2900', ' ff2f00', ' ff3500', ' ff3b00', ' ff4100', ' ff4700', ' ff4d00', ' ff5300', ' ff5900', ' ff5f00', ' ff6400', ' ff6a00', ' ff7000', ' ff7600', ' ff7c00', ' ff8200', ' ff8800', ' ff8e00', ' ff9400', ' ff9a00', ' ff9f00', ' ffa500', ' ffab00', ' ffb100', ' ffb700', ' ffbd00', ' ffc300', ' ffc900', ' ffcf00', ' ffd500', ' ffdb00', ' ffe000', ' ffe600', ' ffec00', ' fef100', ' fcf500', ' faf900', ' f8fd00', ' f4ff00', ' eeff00', ' e8ff00', ' e2ff00', ' ddff00', ' d7ff00', ' d1ff00', ' cbff00', ' c5ff00', ' bfff00', ' b9ff00', ' b3ff00', ' adff00', ' a7ff00', ' a2ff00', ' 9cff00', ' 96ff00', ' 90ff00', ' 8aff00', ' 84ff00', ' 7eff00', ' 78ff00', ' 72ff00', ' 6cff00', ' 66ff00', ' 61ff00', ' 5bff00', ' 55ff00', ' 4fff00', ' 49ff00', ' 43ff00', ' 3dff00', ' 37ff00', ' 31ff00', ' 2bff00', ' 25ff00', ' 20ff00', ' 1aff00', ' 14ff00', ' 0eff00', ' 08ff00', ' 06ff04', ' 04ff08', ' 02ff0c', ' 00ff10', ' 00ff16', ' 00ff1b', ' 00ff21', ' 00ff27', ' 00ff2d', ' 00ff33', ' 00ff39', ' 00ff3f', ' 00ff45', ' 00ff4b', ' 00ff51', ' 00ff57', ' 00ff5c', ' 00ff62', ' 00ff68', ' 00ff6e', ' 00ff74', ' 00ff7a', ' 00ff80', ' 00ff86', ' 00ff8c', ' 00ff92', ' 00ff97', ' 00ff9d', ' 00ffa3', ' 00ffa9', ' 00ffaf', ' 00ffb5', ' 00ffbb', ' 00ffc1', ' 00ffc7', ' 00ffcd', ' 00ffd3', ' 00ffd8', ' 00ffde', ' 00ffe4', ' 00ffea', ' 00fff0', ' 00fff6', ' 00fffc', ' 00fcff', ' 00f6ff', ' 00f0ff', ' 00eaff', ' 00e5ff', ' 00dfff', ' 00d9ff', ' 00d3ff', ' 00cdff', ' 00c7ff', ' 00c1ff', ' 00bbff', ' 00b5ff', ' 00afff', ' 00aaff', ' 00a4ff', ' 009eff', ' 0098ff', ' 0092ff', ' 008cff', ' 0086ff', ' 0080ff', ' 007aff', ' 0074ff', ' 006eff', ' 0069ff', ' 0063ff', ' 005dff', ' 0057ff', ' 0051ff', ' 004bff', ' 0045ff', ' 003fff', ' 0039ff', ' 0033ff', ' 002dff', ' 0028ff', ' 0022ff', ' 001cff', ' 0016ff', ' 0010ff', ' 020cff', ' 0408ff', ' 0604ff', ' 0800ff', ' 0e00ff', ' 1300ff', ' 1900ff', ' 1f00ff', ' 2500ff', ' 2b00ff', ' 3100ff', ' 3700ff', ' 3d00ff', ' 4300ff', ' 4900ff', ' 4f00ff', ' 5400ff', ' 5a00ff', ' 6000ff', ' 6600ff', ' 6c00ff', ' 7200ff', ' 7800ff', ' 7e00ff', ' 8400ff', ' 8a00ff', ' 9000ff', ' 9500ff', ' 9b00ff', ' a100ff', ' a700ff', ' ad00ff', ' b300ff', ' b900ff', ' bf00ff', ' c500ff', ' cb00ff', ' d000ff', ' d600ff', ' dc00ff', ' e200ff', ' e800ff', ' ee00ff', ' f400ff', ' f800fd', ' fa00f9', ' fc00f5', ' fe00f1', ' ff00ed', ' ff00e7', ' ff00e1', ' ff00db', ' ff00d5', ' ff00cf', ' ff00c9', ' ff00c3', ' ff00bd', ' ff00b7', ' ff00b1', ' ff00ac', ' ff00a6', ' ff00a0', ' ff009a', ' ff0094', ' ff008e', ' ff0088', ' ff0082', ' ff007c', ' ff0076', ' ff0071', ' ff006b', ' ff0065', ' ff005f', ' ff0059', ' ff0053', ' ff004d', ' ff0047', ' ff0041', ' ff003b', ' ff0035', ' ff0030', ' ff002a', ' ff0024', ' ff001e', ' ff0018'], 'min': 0.0, 'max': 3000.0, 'opacity': 1.0}


        #判断是哪个传感器，用不同的反演公式
        if sys_id[11] == '5':
            ssc = l5_ssc(ocean).visualize(**vis_params)

        elif sys_id[11] == '7':
            ssc = l7_ssc(ocean).visualize(**vis_params)

        elif sys_id[11] == '8':
            ssc = l8_ssc(ocean).visualize(**vis_params)

    #     Map.addLayer(ssc,{},'ssc')

        # 将陆地设置为0，方便显示
        land = land.where(land,0)
        land = land.visualize(**vis_params0)
    #     Map.addLayer(land,{},'land')

        #为了便于成图陆地海洋合并成一张图             
        global blend 
        blend = land.blend(ssc)
    #     Map.addLayer(blend,{},'blend')


        SSCs.append(blend)

    collection = ee.ImageCollection(SSCs)

    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)

    # 定义动图参数
    video_args = {
        'dimensions': 600,
        'region': roi,
        'framePerSecond': speed0,
        'bands': ['vis-red','vis-green','vis-blue'],
        'min': 0,
        'max': 100,
        'gamma': [1,1,1]
    }

    vv = collection.getVideoThumbURL(video_args)
    geemap.download_from_url(vv, out_dir=work_dir, unzip = False, out_file_name = "test001.gif")

    out_gif = os.path.join(work_dir, "test001.gif")

    geemap.add_text_to_gif(out_gif, out_gif, xy=('3%', '5%'), text_sequence=start_year, font_size=30, font_color=fontcolor, add_progress_bar=False)

    label = title0
    geemap.add_text_to_gif(out_gif, out_gif, xy=('2%', '88%'), text_sequence=label, font_size=30, font_color=fontcolor, progress_bar_color='cyan')
    with out:
        geemap.show_image(out_gif)
    out
    
# 增加点击事件
static_btn.on_click(submit_clicked)
image_btn.on_click(show_image)
ssc_btn.on_click(show_ssc)
# plot_btn.on_click(plt_show)
# sc_btn.on_click(data_import)
# yanzheng_btn.on_click(yanzheng)
exportimg_btn.on_click(eximg)
exportssc_btn.on_click(exssc)
zero_btn.on_click(map_zero)
# seletpath_btn.on_click(select_path)
imagegif_btn.on_click(imagegif)
sscgif_btn.on_click(sscgif)
chla_btn.on_click(show_chla)
wind_btn.on_click(show_wind)